# MLOps Events Data Cleaning & Deduplication

This notebook performs comprehensive data cleaning on the MLOps events CSV dataset. It identifies and removes duplicates based on multiple strategies (talk title, YouTube links, full row), normalizes text fields, extracts canonical YouTube IDs, and produces a clean dataset ready for enrichment and database ingestion.

## Initial Duplicate Analysis

### Find Key-Based Duplicates

Identify duplicate groups based on the composite key: Talk Title + Full Name + YouTube Link.

In [ ]:
import pandas as pd

# Load the CSV
file_path = "/content/mlops-events.csv"  # adjust if needed
df = pd.read_csv(file_path)

# Find duplicate groups
duplicate_groups = (
    df.groupby(["Talk Title", "Full Name", "YouTube Link"])
    .size()
    .reset_index(name="Count")
)

# Keep only duplicates
duplicate_groups = duplicate_groups[duplicate_groups["Count"] > 1]

# For each duplicate group, get the row indices
for _, row in duplicate_groups.iterrows():
    talk, name, yt, count = row["Talk Title"], row["Full Name"], row["YouTube Link"], row["Count"]
    indices = df[
        (df["Talk Title"] == talk) &
        (df["Full Name"] == name) &
        (df["YouTube Link"] == yt)
    ].index.tolist()

    print(f"\nDuplicate group: [Talk Title='{talk}', Full Name='{name}', YouTube Link='{yt}']")
    print(f"Row indices: {indices} (Total: {count})")

# Optionally, show summary at the end
print("\nNumber of duplicate groups:", len(duplicate_groups))
print("Total rows involved in duplicates:", duplicate_groups["Count"].sum())



Duplicate group: [Talk Title='AI Tools Under Control: Keeping Your Agents Secure and Reliable', Full Name='Bar Chen', YouTube Link='https://youtu.be/poqhv4hPTpA?si=wDHbtOf6DGPVJfvl']
Row indices: [61, 413] (Total: 2)

Duplicate group: [Talk Title='Agentic AI: Learning Iteratively, Acting Autonomously', Full Name='Fatma Tarlaci', YouTube Link='https://youtu.be/_k8sPizcqUg?si=nQFRnoSd3voCuvSC']
Row indices: [49, 401] (Total: 2)

Duplicate group: [Talk Title='Build with Mistral', Full Name='Sophia  Yang', YouTube Link='https://youtu.be/_IM53bMowlQ?si=TkQ_sbpgcShA9mOD']
Row indices: [59, 411] (Total: 2)

Duplicate group: [Talk Title='Building AI Infrastructure for the GenAI Wave', Full Name='Shreya Rajpal', YouTube Link='https://youtu.be/Se9_38V2TPA?si=TSU-oQAASiQ5eAG3']
Row indices: [65, 417] (Total: 2)

Duplicate group: [Talk Title='Building Agentic and Multi-Agent Systems with LangGraph (Pt. 2)', Full Name='Greg Loughnane, Chris Alexiuk', YouTube Link='https://youtu.be/uPuoysjaCbw?si=T

### Full Row Duplicates

Check for complete duplicates across all columns.

In [ ]:
import pandas as pd

# Load the CSV
file_path = "/content/mlops-events.csv"
df = pd.read_csv(file_path)

# Find full duplicates (across all columns)
duplicate_rows = df[df.duplicated(keep=False)]

# Group them by all column values
grouped = duplicate_rows.groupby(list(df.columns)).apply(lambda x: x.index.tolist())

# Print each group with row indices
for values, indices in grouped.items():
    print(f"\nDuplicate group (all columns identical):")
    print(f"Row indices: {indices}")
    print(f"Values: {values}")

# Summary
print("\nNumber of full-duplicate groups:", len(grouped))
print("Total rows involved in full duplicates:", len(duplicate_rows))



Duplicate group (all columns identical):
Row indices: Series([], Name: Full Name, dtype: object)
Values: Full Name

Duplicate group (all columns identical):
Row indices: Series([], Name: Company Name, dtype: object)
Values: Company Name

Duplicate group (all columns identical):
Row indices: Series([], Name: Job Title, dtype: object)
Values: Job Title

Duplicate group (all columns identical):
Row indices: Series([], Name: Talk Title, dtype: object)
Values: Talk Title

Duplicate group (all columns identical):
Row indices: Series([], Name: Abstract, dtype: object)
Values: Abstract

Duplicate group (all columns identical):
Row indices: Series([], Name: What You'll Learn, dtype: object)
Values: What You'll Learn

Duplicate group (all columns identical):
Row indices: Series([], Name: Prerequiste Knowledge (if required), dtype: object)
Values: Prerequiste Knowledge (if required)

Duplicate group (all columns identical):
Row indices: Series([], Name: Track, dtype: object)
Values: Track

Dupli

/tmp/ipython-input-913191186.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = duplicate_rows.groupby(list(df.columns)).apply(lambda x: x.index.tolist())


### Inspect Differing Fields

For duplicate groups, identify which fields have different values to understand data inconsistencies.

In [ ]:
import pandas as pd

# Load the CSV
file_path = "/content/mlops-events.csv"
df = pd.read_csv(file_path)

# Find duplicate groups
duplicate_groups = (
    df.groupby(["Talk Title", "Full Name", "YouTube Link"])
    .size()
    .reset_index(name="Count")
)
duplicate_groups = duplicate_groups[duplicate_groups["Count"] > 1]

# Inspect duplicates for differences
for _, row in duplicate_groups.iterrows():
    talk, name, yt = row["Talk Title"], row["Full Name"], row["YouTube Link"]
    dup_rows = df[
        (df["Talk Title"] == talk) &
        (df["Full Name"] == name) &
        (df["YouTube Link"] == yt)
    ].copy()

    print(f"\nDuplicate group: [Talk Title='{talk}', Full Name='{name}', YouTube Link='{yt}']")
    print(f"Row indices: {dup_rows.index.tolist()} (Total: {len(dup_rows)})")

    # Compare values column by column
    differences = {}
    for col in df.columns:
        if len(dup_rows[col].unique()) > 1:  # if column has more than one unique value in this group
            differences[col] = dup_rows[col].tolist()

    if differences:
        print("Fields that differ:")
        for col, vals in differences.items():
            print(f"  {col}: {vals}")
    else:
        print("  No differing fields — rows are exact duplicates.")



Duplicate group: [Talk Title='AI Tools Under Control: Keeping Your Agents Secure and Reliable', Full Name='Bar Chen', YouTube Link='https://youtu.be/poqhv4hPTpA?si=wDHbtOf6DGPVJfvl']
Row indices: [61, 413] (Total: 2)
  No differing fields — rows are exact duplicates.

Duplicate group: [Talk Title='Agentic AI: Learning Iteratively, Acting Autonomously', Full Name='Fatma Tarlaci', YouTube Link='https://youtu.be/_k8sPizcqUg?si=nQFRnoSd3voCuvSC']
Row indices: [49, 401] (Total: 2)
  No differing fields — rows are exact duplicates.

Duplicate group: [Talk Title='Build with Mistral', Full Name='Sophia  Yang', YouTube Link='https://youtu.be/_IM53bMowlQ?si=TkQ_sbpgcShA9mOD']
Row indices: [59, 411] (Total: 2)
  No differing fields — rows are exact duplicates.

Duplicate group: [Talk Title='Building AI Infrastructure for the GenAI Wave', Full Name='Shreya Rajpal', YouTube Link='https://youtu.be/Se9_38V2TPA?si=TSU-oQAASiQ5eAG3']
Row indices: [65, 417] (Total: 2)
  No differing fields — rows are e

## Advanced Normalization & Diagnosis

### Normalize Text for Fair Comparison

Create cleaned views of data with normalized whitespace, Unicode characters (NFKC), and hidden characters exposed.

In [ ]:
import pandas as pd
import numpy as np

# === Load ===
file_path = "/content/mlops-events.csv"  # adjust as needed
df = pd.read_csv(file_path)

# === Create two views ===
# raw_df: untouched; clean_df: normalized strings for fair comparison
raw_df = df.copy()

clean_df = df.copy()
for c in clean_df.select_dtypes(include=["object"]).columns:
    # normalize strings: strip, collapse internal whitespace, normalize case for urls/titles/names as desired
    clean_df[c] = clean_df[c].astype(str).str.replace(r"\s+", " ", regex=True).str.strip()

# === 1) Whole-row duplicates (exact) ===
whole_row_dupe_mask_raw   = raw_df.duplicated(keep=False)
whole_row_dupe_mask_clean = clean_df.duplicated(keep=False)

print("Exact duplicate rows (raw):", int(whole_row_dupe_mask_raw.sum()))
print("Exact duplicate rows (cleaned):", int(whole_row_dupe_mask_clean.sum()))

# === 2) Key-duplicates on the trio ===
KEY = ["Talk Title", "Full Name", "YouTube Link"]
missing_keys = [k for k in KEY if k not in df.columns]
if missing_keys:
    raise ValueError(f"Missing expected columns: {missing_keys}")

key_dupe_counts = clean_df.groupby(KEY).size().reset_index(name="Count")
key_dupe_groups = key_dupe_counts[key_dupe_counts["Count"] > 1]

print("Duplicate key groups (same trio):", len(key_dupe_groups))
print("Total rows in key-duplicate groups:", int(key_dupe_groups["Count"].sum()))

# === 3) For each key-duplicate group, check if rows are fully identical (across ALL columns) ===
def row_signature(row):
    # tuple of all columns in order; NaN-safe
    return tuple((None if pd.isna(v) else v) for v in row.tolist())

fully_identical_groups = []
partly_different_groups = []

for _, grp_key in key_dupe_groups[KEY].iterrows():
    mask = (clean_df[KEY] == grp_key.values).all(axis=1)
    grp = clean_df.loc[mask]

    # Build signatures across ALL columns for each row
    sigs = grp.apply(row_signature, axis=1)
    unique_sigs = sigs.drop_duplicates()

    if len(unique_sigs) == 1:
        fully_identical_groups.append(tuple(grp_key.values))
    else:
        partly_different_groups.append(tuple(grp_key.values))

print("Key-duplicate groups that are full-row identical (cleaned view):", len(fully_identical_groups))
print("Key-duplicate groups with some differing fields:", len(partly_different_groups))

# === 4) Show diagnostics for differences (including hidden chars) ===
def show_hidden(s):
    if pd.isna(s):
        return "<NaN>"
    return repr(str(s))  # exposes escaped whitespace like '\xa0', '\n', etc.

def explain_group(triple, max_rows=10):
    talk, name, yt = triple
    mask_raw   = (raw_df["Talk Title"] == talk) & (raw_df["Full Name"] == name) & (raw_df["YouTube Link"] == yt)
    mask_clean = (clean_df["Talk Title"] == talk) & (clean_df["Full Name"] == name) & (clean_df["YouTube Link"] == yt)

    raw_rows   = raw_df.loc[mask_raw]
    clean_rows = clean_df.loc[mask_clean]

    print("\n=== Group ===")
    print(f"Talk Title: {talk}")
    print(f"Full Name:  {name}")
    print(f"YouTube:    {yt}")
    print("Row indices (raw):  ", raw_rows.index.tolist())
    print("Row indices (clean):", clean_rows.index.tolist())

    # Column-by-column unique values (cleaned)
    diffs = {}
    for c in clean_df.columns:
        vals = clean_rows[c]
        uniq = pd.unique(vals)
        if len(uniq) > 1:
            diffs[c] = [show_hidden(x) for x in uniq[:max_rows]]

    if diffs:
        print("Differing fields (cleaned view):")
        for c, vals in diffs.items():
            print(f"  - {c}: {vals}")
    else:
        print("No differing fields in cleaned view (full-row identical after normalization).")

# Example: print details for a few groups (both categories)
for triple in fully_identical_groups[:3]:
    explain_group(triple)

for triple in partly_different_groups[:3]:
    explain_group(triple)


Exact duplicate rows (raw): 103
Exact duplicate rows (cleaned): 103
Duplicate key groups (same trio): 30
Total rows in key-duplicate groups: 113
Key-duplicate groups that are full-row identical (cleaned view): 29
Key-duplicate groups with some differing fields: 1

=== Group ===
Talk Title: AI Tools Under Control: Keeping Your Agents Secure and Reliable
Full Name:  Bar Chen
YouTube:    https://youtu.be/poqhv4hPTpA?si=wDHbtOf6DGPVJfvl
Row indices (raw):   [61, 413]
Row indices (clean): [61, 413]
No differing fields in cleaned view (full-row identical after normalization).

=== Group ===
Talk Title: Agentic AI: Learning Iteratively, Acting Autonomously
Full Name:  Fatma Tarlaci
YouTube:    https://youtu.be/_k8sPizcqUg?si=nQFRnoSd3voCuvSC
Row indices (raw):   [49, 401]
Row indices (clean): [49, 401]
No differing fields in cleaned view (full-row identical after normalization).

=== Group ===
Talk Title: Build with Mistral
Full Name:  Sophia Yang
YouTube:    https://youtu.be/_IM53bMowlQ?si=T

## Deterministic Deduplication

### Build Canonical Unique Keys

Create stable unique keys using talk title + speaker name + YouTube ID, then implement scoring-based deduplication (prefer rows with more data, YouTube ID present, longer abstracts).

In [ ]:
# Colab-ready cell for your schema (uses "YouTube Link")
import pandas as pd
import numpy as np
import re
import unicodedata
from urllib.parse import urlparse, parse_qs
from hashlib import blake2b
import json

# ===== 0) Load CSV =====
CSV_PATH = "/content/mlops-events.csv"  # change if needed
df = pd.read_csv(CSV_PATH)

expected_cols = [
    "Full Name", "Company Name", "Job Title", "Talk Title", "Abstract",
    "What You'll Learn", "Prerequiste Knowledge (if required)", "Track",
    "Technical Level (1-7)", "Category 1", "Top 3 keywords (in order)",
    "Bio", "YouTube Link", "Relevant Industries",
    "What is Unique about your session", "Event"
]
missing = [c for c in expected_cols if c not in df.columns]
if missing:
    raise ValueError(f"Missing columns: {missing}")

# ===== 1) Normalization helpers =====
def norm_text(s, lower=True):
    """Unicode NFKC, strip, collapse whitespace; optional lowercase."""
    if pd.isna(s):
        return None
    s = unicodedata.normalize("NFKC", str(s))
    s = re.sub(r"\s+", " ", s).strip()
    return s.lower() if lower else s

def extract_youtube_id(url):
    """Extract a YouTube video ID from common URL formats."""
    if pd.isna(url) or str(url).strip() == "":
        return None
    url = unicodedata.normalize("NFKC", str(url)).strip()

    # youtu.be/<id>
    m = re.match(r"^https?://(?:www\.)?youtu\.be/([A-Za-z0-9_-]{6,})", url)
    if m:
        return m.group(1)

    # youtube.com variants
    try:
        p = urlparse(url)
        if "youtube.com" in p.netloc or "m.youtube.com" in p.netloc:
            q = parse_qs(p.query)
            if "v" in q and len(q["v"]) > 0:
                return q["v"][0]
            # /shorts/<id>
            m = re.match(r"^/shorts/([A-Za-z0-9_-]{6,})", p.path)
            if m:
                return m.group(1)
            # /embed/<id>
            m = re.match(r"^/embed/([A-Za-z0-9_-]{6,})", p.path)
            if m:
                return m.group(1)
    except Exception:
        pass

    # Fallback: guess an 11-char token
    m = re.search(r"([A-Za-z0-9_-]{11})(?:\b|$)", url)
    return m.group(1) if m else None

# ===== 2) Canonical fields for uniqueness =====
df["_talk_norm"] = df["Talk Title"].apply(lambda x: norm_text(x, lower=True))
df["_name_norm"] = df["Full Name"].apply(lambda x: norm_text(x, lower=True))
df["_yt_id"]     = df["YouTube Link"].apply(extract_youtube_id)

# ===== 3) Stable unique key (talk|name|ytid) =====
def make_key(talk, name, ytid):
    triple = f"{talk or ''}||{name or ''}||{ytid or ''}"
    return blake2b(triple.encode("utf-8"), digest_size=10).hexdigest()

df["_uniq_key"] = [make_key(t, n, y) for t, n, y in zip(df["_talk_norm"], df["_name_norm"], df["_yt_id"])]

# ===== 4) Diagnose what’s “wrong” (clashing groups + differing fields) =====
key_counts = df.groupby("_uniq_key").size().rename("Count")
clashing_keys = key_counts[key_counts > 1].index.tolist()

def differing_fields(group):
    """Columns whose values differ within given group."""
    diffs = {}
    for c in df.columns:
        vals = group[c]
        uniq = {repr(x) for x in vals}  # repr exposes hidden whitespace/characters
        if len(uniq) > 1:
            diffs[c] = list(uniq)
    return diffs

print(f"Duplicate groups by key: {len(clashing_keys)}")
if clashing_keys:
    print("Showing up to 5 clashing groups (with differing fields):\n")
    for k in clashing_keys[:5]:
        grp = df[df["_uniq_key"] == k]
        diffs = differing_fields(grp)
        print(f"key={k} | size={len(grp)} | row_indices={grp.index.tolist()}")
        if diffs:
            print("  Differing fields:")
            for col, reps in diffs.items():
                # truncate long reps to keep console readable
                preview = [r if len(r) < 160 else r[:157] + "...'" for r in reps]
                print(f"   - {col}: {preview}")
        else:
            print("  No differing fields (fully identical rows).")
        print("-"*80)

# ===== 5) Deterministic dedupe policy =====
# Prefer rows with: (a) most non-null fields, (b) longest Abstract, (c) YouTube ID present, (d) first seen
non_null_counts = df.notna().sum(axis=1)
has_ytid = df["_yt_id"].notna().astype(int)
abstract_len = df["Abstract"].fillna("").str.len()

# Higher score is better
df["_score"] = (
    non_null_counts * 1.0
    + abstract_len * 1e-6         # tiny tie-breaker
    + has_ytid * 0.1              # prefer rows that resolve to a YT id
)

# Keep the highest-score row per key (break ties by earliest index)
dedup = (
    df.sort_values(["_uniq_key", "_score", df.index.name if df.index.name else "Full Name"], ascending=[True, False, True])
      .groupby("_uniq_key", as_index=False)
      .head(1)
      .copy()
)

print(f"\nOriginal rows: {len(df)}")
print(f"Unique rows after dedupe: {len(dedup)} (removed {len(df) - len(dedup)})")

# ===== 6) Build your per-row objects =====
def build_object(row):
    return {
        "full_name": row["Full Name"],
        "company_name": row["Company Name"],
        "job_title": row["Job Title"],
        "talk_title": row["Talk Title"],
        "abstract": row["Abstract"],
        "what_youll_learn": row["What You'll Learn"],
        "prerequisite_knowledge": row["Prerequiste Knowledge (if required)"],
        "track": row["Track"],
        "technical_level": row["Technical Level (1-7)"],
        "category_1": row["Category 1"],
        "top_3_keywords": row["Top 3 keywords (in order)"],
        "bio": row["Bio"],
        "youtube_link": row["YouTube Link"],
        "youtube_id": row["_yt_id"],
        "relevant_industries": row["Relevant Industries"],
        "unique_session": row["What is Unique about your session"],
        "event": row["Event"],
        "unique_key": row["_uniq_key"],
    }

objects = [build_object(r) for _, r in dedup.iterrows()]
print(f"Built {len(objects)} objects.")

# ===== 7) (Optional) Save cleaned CSV and JSON =====
OUT_CSV  = "/content/mlops-events.cleaned.csv"
OUT_JSON = "/content/mlops-events.objects.json"

# Drop helper columns before writing cleaned CSV
helper_cols = [c for c in dedup.columns if c.startswith("_")]
dedup_clean = dedup.drop(columns=helper_cols)
dedup_clean.to_csv(OUT_CSV, index=False)

with open(OUT_JSON, "w", encoding="utf-8") as f:
    json.dump(objects, f, ensure_ascii=False, indent=2)

print("Wrote:", OUT_CSV, "and", OUT_JSON)


Duplicate groups by key: 31
Showing up to 5 clashing groups (with differing fields):

key=0b5bab7e69fc0f337894 | size=2 | row_indices=[57, 409]
  No differing fields (fully identical rows).
--------------------------------------------------------------------------------
key=0f8ca83c95553d53a797 | size=2 | row_indices=[68, 420]
  No differing fields (fully identical rows).
--------------------------------------------------------------------------------
key=1785ea18584c46433ddb | size=2 | row_indices=[53, 405]
  No differing fields (fully identical rows).
--------------------------------------------------------------------------------
key=18d41ecec98bfca7de3a | size=2 | row_indices=[49, 401]
  No differing fields (fully identical rows).
--------------------------------------------------------------------------------
key=18db96b2224bd2f868c1 | size=2 | row_indices=[61, 413]
  No differing fields (fully identical rows).
----------------------------------------------------------------------

## Validation Checks

### Count Unique Values

Verify uniqueness across key fields after initial deduplication.

In [ ]:
import pandas as pd

# Load the cleaned file you wrote in the previous step
df = pd.read_csv("/content/mlops-events.cleaned.csv")

# Count uniques
unique_full_names   = df["Full Name"].nunique()
unique_talk_titles  = df["Talk Title"].nunique()
unique_youtube_urls = df["YouTube Link"].nunique()

print("Unique Full Names   :", unique_full_names)
print("Unique Talk Titles  :", unique_talk_titles)
print("Unique YouTube Links:", unique_youtube_urls)


Unique Full Names   : 311
Unique Talk Titles  : 340
Unique YouTube Links: 280


### Remove Talk Title Duplicates

Drop duplicate talk titles, keeping only the first occurrence.

In [ ]:
import pandas as pd

# Load the cleaned file
path = "/content/mlops-events.cleaned.csv"
df = pd.read_csv(path)

# Drop duplicates by Talk Title, keeping the first occurrence
df_dedup = df.drop_duplicates(subset=["Talk Title"], keep="first")

print("Rows before:", len(df))
print("Rows after :", len(df_dedup))
print("Unique Talk Titles:", df_dedup["Talk Title"].nunique())

# Overwrite the same file
df_dedup.to_csv(path, index=False)
print(f"Overwritten file saved at {path}")


Rows before: 341
Rows after : 341
Unique Talk Titles: 340
Overwritten file saved at /content/mlops-events.cleaned.csv


Show remaining duplicate talk titles (if any) for manual review.

In [ ]:
# Find titles that occur more than once
dupe_titles = df["Talk Title"][df["Talk Title"].duplicated(keep=False)]

# Show all rows with those titles
dupe_rows = df[df["Talk Title"].isin(dupe_titles)]
print(dupe_rows[["Full Name", "Talk Title", "YouTube Link"]])


Empty DataFrame
Columns: [Full Name, Talk Title, YouTube Link]
Index: []


## YouTube URL Normalization

### Identify YouTube Link Duplicates

Find duplicate YouTube URLs before normalization.

In [ ]:
import pandas as pd

# Load the cleaned file
df = pd.read_csv("/content/mlops-events.cleaned.csv")

# Find duplicates in YouTube Link column
duplicate_urls = (
    df.groupby("YouTube Link")
    .size()
    .reset_index(name="Count")
    .query("Count > 1")
)

print("Number of duplicate YouTube URLs:", len(duplicate_urls))
print("Total rows involved in duplicates:", duplicate_urls["Count"].sum())

# Show details for each duplicate group
for _, row in duplicate_urls.iterrows():
    url = row["YouTube Link"]
    count = row["Count"]
    indices = df[df["YouTube Link"] == url].index.tolist()
    print(f"\nYouTube URL: {url}")
    print(f"Appears {count} times")
    print(f"Row indices: {indices}")


Number of duplicate YouTube URLs: 0
Total rows involved in duplicates: 0


Check potential duplicates after basic normalization (lowercase, strip whitespace).

In [ ]:
# Check how many rows vs uniques again
print("Total rows:", len(df))
print("Unique YouTube Links:", df["YouTube Link"].nunique())

# Show potential duplicates (ignoring case and spaces)
df["YouTube_norm"] = df["YouTube Link"].str.strip().str.lower()

dupes = df[df.duplicated(subset=["YouTube_norm"], keep=False)]
print("Potential duplicates after normalization:", len(dupes))

# Group and show
dupes_grouped = dupes.groupby("YouTube_norm").agg({
    "YouTube Link": list,
    "Full Name": list,
    "Talk Title": list
})
print(dupes_grouped.head(10))


Total rows: 341
Unique YouTube Links: 280
Potential duplicates after normalization: 61
Empty DataFrame
Columns: [YouTube Link, Full Name, Talk Title]
Index: []


### Extract & Canonicalize YouTube IDs

Robust extraction of YouTube video IDs from various URL formats (youtu.be, youtube.com, shorts, embed). Create canonical links in standard format.

In [ ]:
import pandas as pd
import re, unicodedata
from urllib.parse import urlparse, parse_qs

# --- 1) Load ---
path = "/content/mlops-events.cleaned.csv"
df = pd.read_csv(path)

# --- 2) Robust extractor for YouTube video IDs ---
def extract_youtube_id(url):
    if pd.isna(url) or str(url).strip() == "":
        return None
    url = unicodedata.normalize("NFKC", str(url)).strip()

    # youtu.be/<id>
    m = re.match(r"^https?://(?:www\.)?youtu\.be/([A-Za-z0-9_-]{6,})", url)
    if m:
        return m.group(1)

    # youtube.com variants
    try:
        p = urlparse(url)
        host = (p.netloc or "").lower()
        if "youtube.com" in host or "m.youtube.com" in host:
            q = parse_qs(p.query)
            if "v" in q and len(q["v"]) > 0:
                return q["v"][0]
            # /shorts/<id>
            m = re.match(r"^/shorts/([A-Za-z0-9_-]{6,})", p.path)
            if m:
                return m.group(1)
            # /embed/<id>
            m = re.match(r"^/embed/([A-Za-z0-9_-]{6,})", p.path)
            if m:
                return m.group(1)
    except Exception:
        pass

    # Fallback: guess an 11-char token
    m = re.search(r"([A-Za-z0-9_-]{11})(?:\b|$)", url)
    return m.group(1) if m else None

# --- 3) BEFORE stats ---
before_unique = df["YouTube Link"].nunique()
print("Unique YouTube Links (BEFORE):", before_unique)

# --- 4) Compute IDs and canonicalize the link column ---
yt_ids = df["YouTube Link"].apply(extract_youtube_id)
df["YouTube ID"] = yt_ids

# Build canonical link only when we have an ID; otherwise keep original
def canonical_link(vid, raw):
    if pd.isna(vid) or not vid:
        return raw  # preserve whatever was there if we can't parse
    return f"https://www.youtube.com/watch?v={vid}"

df["YouTube Link"] = [canonical_link(v, r) for v, r in zip(df["YouTube ID"], df["YouTube Link"])]

# --- 5) AFTER stats + quick sanity check on duplicates ---
after_unique = df["YouTube Link"].nunique()
print("Unique YouTube Links (AFTER):", after_unique)

# Show duplicate URLs (should now truly group together)
dupe_urls = (
    df.groupby("YouTube Link")
      .size()
      .reset_index(name="Count")
      .query("Count > 1")
      .sort_values("Count", ascending=False)
)
print("Duplicate YouTube URLs (AFTER normalization):", len(dupe_urls))
if not dupe_urls.empty:
    for _, row in dupe_urls.head(10).iterrows():  # print up to 10 groups for brevity
        url = row["YouTube Link"]
        idxs = df.index[df["YouTube Link"] == url].tolist()
        print(f"{url} -> {row['Count']} rows | indices: {idxs}")

# --- 6) Overwrite the cleaned file ---
df.to_csv(path, index=False)
print(f"Overwritten: {path}")


Unique YouTube Links (BEFORE): 280
Unique YouTube Links (AFTER): 280
Duplicate YouTube URLs (AFTER normalization): 0
Overwritten: /content/mlops-events.cleaned.csv


Verify duplicate URLs after normalization (should be properly grouped now).

In [ ]:
import pandas as pd

# Load the cleaned file
df = pd.read_csv("/content/mlops-events.cleaned.csv")

# Find duplicates in YouTube Link column
duplicate_urls = (
    df.groupby("YouTube Link")
    .size()
    .reset_index(name="Count")
    .query("Count > 1")
)

print("Number of duplicate YouTube URLs:", len(duplicate_urls))
print("Total rows involved in duplicates:", duplicate_urls["Count"].sum())

# Show details for each duplicate group
for _, row in duplicate_urls.iterrows():
    url = row["YouTube Link"]
    count = row["Count"]
    indices = df[df["YouTube Link"] == url].index.tolist()
    print(f"\nYouTube URL: {url}")
    print(f"Appears {count} times")
    print(f"Row indices: {indices}")


Number of duplicate YouTube URLs: 0
Total rows involved in duplicates: 0


## Final Statistics

### Comprehensive Link Analysis

Calculate total rows, null/non-null YouTube links, unique counts, and identify remaining duplicates.

In [ ]:
import pandas as pd

df = pd.read_csv("/content/mlops-events.cleaned.csv")

total_rows = len(df)
non_null    = df["YouTube Link"].notna().sum()
nulls       = df["YouTube Link"].isna().sum()
uniques     = df["YouTube Link"].nunique()                # dropna=True (default)
uniques_all = df["YouTube Link"].nunique(dropna=False)    # counts NaN as a distinct value

print(f"Total rows: {total_rows}")
print(f"Non-null YouTube Links: {non_null}")
print(f"Null (missing) YouTube Links: {nulls}")
print(f"Unique YouTube Links (non-null only): {uniques}")
print(f"Unique YouTube Links (counting NaN as a category): {uniques_all}")

# Real duplicate URLs among NON-NULL values
dupe_urls = (
    df[df["YouTube Link"].notna()]
      .groupby("YouTube Link")
      .size()
      .reset_index(name="Count")
      .query("Count > 1")
      .sort_values("Count", ascending=False)
)

print("\nDuplicate non-null YouTube URLs:", len(dupe_urls))
if not dupe_urls.empty:
    for _, row in dupe_urls.iterrows():
        url = row["YouTube Link"]
        idxs = df.index[df["YouTube Link"] == url].tolist()
        print(f"{url} -> {row['Count']} rows | indices: {idxs}")

# OPTIONAL: if you want to also consider missing links as a single "duplicate group"
# (this is unusual, but sometimes useful), fill NaN with a sentinel and group:
df["_YouTube_Link_with_NA"] = df["YouTube Link"].fillna("<NA>")
dupe_including_na = (
    df.groupby("_YouTube_Link_with_NA")
      .size()
      .reset_index(name="Count")
      .query("Count > 1")
      .sort_values("Count", ascending=False)
)
print("\nGroups with Count>1 when treating NaN as '<NA>':")
print(dupe_including_na.head(10))


Total rows: 341
Non-null YouTube Links: 280
Null (missing) YouTube Links: 61
Unique YouTube Links (non-null only): 280
Unique YouTube Links (counting NaN as a category): 281

Duplicate non-null YouTube URLs: 0

Groups with Count>1 when treating NaN as '<NA>':
  _YouTube_Link_with_NA  Count
0                  <NA>     61


## Remove Missing YouTube Links

Drop rows where YouTube Link is missing or empty to ensure all records have valid video data.

In [ ]:
import pandas as pd

# Path to your cleaned file
path = "/content/mlops-events.cleaned.csv"

# Load the file
df = pd.read_csv(path)

# Drop rows where YouTube Link is missing/empty
df_no_empty = df.dropna(subset=["YouTube Link"])

print("Rows before:", len(df))
print("Rows after :", len(df_no_empty))
print("Unique YouTube Links:", df_no_empty["YouTube Link"].nunique())

# Overwrite the cleaned file
df_no_empty.to_csv(path, index=False)
print(f"File overwritten at {path}")


Rows before: 341
Rows after : 280
Unique YouTube Links: 280
File overwritten at /content/mlops-events.cleaned.csv


## Final Validation

### Unique Talk Titles Count

Confirm final count of unique talk titles in cleaned dataset.

In [ ]:
import pandas as pd

# Load the updated cleaned file
df = pd.read_csv("/content/mlops-events.cleaned.csv")

# Count unique talk titles
unique_talk_titles = df["Talk Title"].nunique()

print("Unique Talk Titles:", unique_talk_titles)


Unique Talk Titles: 280


### Unique Speakers Count

Confirm final count of unique speakers in cleaned dataset.

In [ ]:
import pandas as pd

# Load the updated cleaned file
df = pd.read_csv("/content/mlops-events.cleaned.csv")

# Count unique full names
unique_full_names = df["Full Name"].nunique()

print("Unique Full Names:", unique_full_names)


Unique Full Names: 263
